unlike with get, we only have one special method to set an attribute

In [1]:
class Person:
    
    def __setattr__(self, name, value):
        print("setting instance attribute...")
        return super().__setattr__(name, value)

In [2]:
p = Person()

In [3]:
p.name = "Alex"

setting instance attribute...


In [4]:
p.__dict__

{'name': 'Alex'}

`__setattr__` would not be called for class attribute

In [5]:
Person.class_attr = "test"

In [6]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__setattr__': <function __main__.Person.__setattr__(self, name, value)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'class_attr': 'test'})

we can override `__setattr__` for class attributes in the metaclass

In [7]:
class MyMeta(type):
    def __setattr__(self, name, value):
        print("setting class attribute...")
        return super().__setattr__(name, value)

In [8]:
class Person(metaclass=MyMeta):
    
    def __setattr__(self, name, value):
        print("setting instance attribute...")
        return super().__setattr__(name, value)

In [9]:
Person.test = "test"

setting class attribute...


if `__setattr__` sets data descriptor, it calles data descriptor method instead. it doesn't put it into the dictionary

In [10]:
class MyNonDataDesc:
    def __get__(self, instance, owner_class):
        print("__get__ called on non-data descriptor.")
        
class MyDataDesc:
    def __get__(self, instance, owner_class):
        print("__get__ called on data descriptor.")
        
    def __set__(self, instance, value):
        print("__set__ called on data descriptor.")

In [11]:
class MyClass:
    non_data_desc = MyNonDataDesc()
    data_desc = MyDataDesc()
    
    def __setattr__(self, name, value):
        print("__setattr__ called.")
        super().__setattr__(name, value)

In [12]:
m = MyClass()

In [13]:
vars(m)

{}

In [14]:
m.data_desc = 100

__setattr__ called.
__set__ called on data descriptor.


In [15]:
m.non_data_desc = 200

__setattr__ called.


In [16]:
vars(m)

{'non_data_desc': 200}

we need to be careful with infinite recursion, so we should call super to avoid it

we want to disallow to set attributes which start with `_`

In [17]:
class MyClass:
    
        
    def __setattr__(self, name, value):
        print("__setattr__ called.")

        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f"This attribute is read-only")
        return super().__setattr__(name, value) 

In [18]:
m = MyClass()

In [19]:
m._test = "test"

__setattr__ called.


AttributeError: This attribute is read-only

In [20]:
m.test = "test"

__setattr__ called.


In [21]:
vars(m)

{'test': 'test'}